<a href="https://colab.research.google.com/github/kimarlee/Tic-Tac-Toe/blob/main/Tic_Tac_Toe_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic-Tac-Toe Move Prediction Project

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version. For example: spark_version = 'spark-3.5.6'
spark_version = 'spark-3.5.6'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 kB in 2s (163 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Import packages
from pyspark.sql import SparkSession  # entry point for working with Spark DataFrames and executing Spark SQL queries
import time                           # time module so we can time our queries.

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# Read the tic_tac_toe_data.csv into a PySpark DataFrame.
from pyspark import SparkFiles   # allows Spark to access files from remote URLs like GitHub
url = "https://raw.githubusercontent.com/kimarlee/Tic-Tac-Toe/refs/heads/main/tic_tac_toe_data.csv"

# Downloads the file - this step makes the file accessible locally within your Spark session — even though it came from a remote URL.
spark.sparkContext.addFile(url)

# Read the data into the dataframe
df = spark.read.csv(SparkFiles.get("tic_tac_toe_data.csv"), sep=",", inferSchema=True, header=True)
df.show()

+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Winner|Move 1-X (Row-Col)|Move 2-O (Row-Col)|Move 3-X (Row-Col)|Move 4-O (Row-Col)|Move 5-X (Row-Col)|Move 6-O (Row-Col)|Move 7-X (Row-Col)|Move 8-O (Row-Col)|Move 9-X (Row-Col)|
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|     X|               0-0|               0-1|               1-0|               0-2|               2-0|               ---|               ---|               ---|               ---|
|     X|               0-0|               0-1|               1-0|               1-1|               2-0|               ---|               ---|               ---|               ---|
|     X|               0-0|               0-1|               1-0|               1-2|               2

In [ ]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView('tictactoe_games')

Clean and Prepare Move Data

In [ ]:
from pyspark.sql.functions import col, when

# Get the list of move columns
move_columns = [c for c in df.columns if c.startswith('Move')]

# Replace NULL values with 9 for all move columns
for column_name in move_columns:
    df = df.withColumn(
        column_name,
        when(col(column_name).isNull(), 9).otherwise(col(column_name))
    )

# Select only the 'Winner' column and the move columns
df = df.select('Winner', *move_columns)

# Show the result
df.show(truncate=False)

# Print the schema to confirm the data types
df.printSchema()

+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Winner|Move 1-X (Row-Col)|Move 2-O (Row-Col)|Move 3-X (Row-Col)|Move 4-O (Row-Col)|Move 5-X (Row-Col)|Move 6-O (Row-Col)|Move 7-X (Row-Col)|Move 8-O (Row-Col)|Move 9-X (Row-Col)|
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|X     |0-0               |0-1               |1-0               |0-2               |2-0               |---               |---               |---               |---               |
|X     |0-0               |0-1               |1-0               |1-1               |2-0               |---               |---               |---               |---               |
|X     |0-0               |0-1               |1-0               |1-2               |2-0             

# Model Baseline Implementation (First Attempt)

In [ ]:
# PySpark ML and core tools
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [ ]:
# Select all columns that represent moves
move_columns = [col for col in df.columns if col.startswith('Move')]

# Replace null values with 'Empty' for all move columns
for column in move_columns:
    df = df.withColumn(column, when(col(column).isNull(), "Empty").otherwise(col(column)))

In [ ]:
# Initialize an empty list to hold the pipeline stages
stages = []

# Convert Winner to numeric
labelIndexer = StringIndexer(inputCol="Winner", outputCol="label")
stages.append(labelIndexer)

# Create StringIndexer and OneHotEncoder for each move column
encoded_move_columns = []

for move_col in move_columns:
    # StringIndexer - convert string categories into numerical indices
    indexer = StringIndexer(inputCol=move_col, outputCol=f"{move_col}_indexed", handleInvalid="keep")
    stages.append(indexer)

    # OneHotEncoder - convert numerical indices into one-hot encoded vectors
    encoder = OneHotEncoder(inputCol=f"{move_col}_indexed", outputCol=f"{move_col}_encoded", handleInvalid="keep")
    stages.append(encoder)

    # Track the encoded columns to be used as features later
    encoded_move_columns.append(f"{move_col}_encoded")

# Assemble features
assembler = VectorAssembler(inputCols=encoded_move_columns, outputCol="features")
stages.append(assembler)

# Create and train the model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)
stages.append(rf)

# Create the pipeline
pipeline = Pipeline(stages=stages)

In [ ]:
# Split the data into training and test sets
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

# Train model
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions
predictions = model.transform(testData)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8014061131947258


In [ ]:
# Get the label mapping
label_mapping = {float(idx): category for idx, category in enumerate(model.stages[0].labels)}
print("Label Mapping:", label_mapping)

Label Mapping: {0.0: 'X', 1.0: 'O', 2.0: '-'}


In [ ]:
# Convert predictions to RDD and initialize MulticlassMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)

# Show confusion matrix
print("Confusion Matrix:")
print(metrics.confusionMatrix())

Confusion Matrix:
DenseMatrix([[39369.,     0.,     0.],
             [ 1529., 21957.,     0.],
             [13668.,     0.,     0.]])


In [ ]:
# Display precision, recall, and F1-score for each class
for label in sorted(label_mapping.keys()):
    print(f"\nClass {label_mapping[label]}:")
    print(f"Precision = {metrics.precision(label)}")
    print(f"Recall = {metrics.recall(label)}")
    print(f"F1 Score = {metrics.fMeasure(label)}")


Class X:
Precision = 0.7214932375471905
Recall = 1.0
F1 Score = 0.8382179166444882

Class O:
Precision = 1.0
Recall = 0.9348973856765733
F1 Score = 0.9663534537772595

Class -:
Precision = 0.0
Recall = 0.0
F1 Score = 0.0


### 🔧 Optimization Next Steps
The initial model performs well in predicting Player X and O wins, achieving high precision and recall for both. However, it completely fails to identify tied games (represented by the "-" class), resulting in zero precision and recall for that category. This clear gap in performance guided our team to further investigate how the model processes and represents ties.

### Clean and Convert Move Columns to Numbers

In [ ]:
# Corrected version to Integers
from pyspark.sql.functions import col, when, split
from pyspark.sql.types import IntegerType

# Get the list of move columns
move_columns = [c for c in df.columns if c.startswith('Move')]

# Function to convert move notation to a single integer
def convert_move(col_name):
    return when(col(col_name) == "---", 9)\
           .otherwise(
               (split(col(col_name), "-")[0].cast("int") * 3 +
                split(col(col_name), "-")[1].cast("int"))
           )

# Apply the conversion to all move columns
for column_name in move_columns:
    df = df.withColumn(
        column_name,
        convert_move(column_name).cast(IntegerType())
    )

# Select only the 'Winner' column and the move columns
df = df.select('Winner', *move_columns)

# Show the result
print("Transformed Data:")
df.show(truncate=False)

# Print the schema to confirm the data types
print("\nUpdated DataFrame Schema:")
df.printSchema()

Transformed Data:
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Winner|Move 1-X (Row-Col)|Move 2-O (Row-Col)|Move 3-X (Row-Col)|Move 4-O (Row-Col)|Move 5-X (Row-Col)|Move 6-O (Row-Col)|Move 7-X (Row-Col)|Move 8-O (Row-Col)|Move 9-X (Row-Col)|
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|X     |0                 |1                 |3                 |2                 |6                 |9                 |9                 |9                 |9                 |
|X     |0                 |1                 |3                 |4                 |6                 |9                 |9                 |9                 |9                 |
|X     |0                 |1                 |3                 |5                

# Second Attempt

In [ ]:
# Get the list of move columns
move_columns = [col for col in df.columns if col.startswith('Move')]

In [ ]:
# Create stages for the pipeline
stages = []

# Convert Winner to numeric
labelIndexer = StringIndexer(inputCol="Winner", outputCol="label")
stages.append(labelIndexer)

# Assemble features
assembler = VectorAssembler(inputCols=move_columns, outputCol="features")
stages.append(assembler)

# Create and train the model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)
stages.append(rf)

# Create the pipeline
pipeline = Pipeline(stages=stages)

In [ ]:
# Split the data into training and test sets
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

# Train model
model = pipeline.fit(trainingData)

In [ ]:
# Make predictions
predictions = model.transform(testData)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8213870339636449


In [ ]:
# Retrieve the original label mapping from the model
label_mapping = {float(idx): category for idx, category in enumerate(model.stages[0].labels)}
print("Label Mapping:", label_mapping)

# Generate the confusion matrix
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)

print("Confusion Matrix:")
print(metrics.confusionMatrix())

Label Mapping: {0.0: 'X', 1.0: 'O', 2.0: '-'}
Confusion Matrix:
DenseMatrix([[39369.,     0.,     0.],
             [    0., 23486.,     0.],
             [13668.,     0.,     0.]])


In [ ]:
# Print out precision, recall, and F1 score for each class
for label in sorted(label_mapping.keys()):
    print(f"\nClass {label_mapping[label]}:")
    print(f"Precision = {metrics.precision(label)}")
    print(f"Recall = {metrics.recall(label)}")
    print(f"F1 Score = {metrics.fMeasure(label)}")


Class X:
Precision = 0.7422931161264777
Recall = 1.0
F1 Score = 0.852087526783975

Class O:
Precision = 1.0
Recall = 1.0
F1 Score = 1.0

Class -:
Precision = 0.0
Recall = 0.0
F1 Score = 0.0


### 🔧 Optimization Next Steps
This second attempt shows a slight improvement in overall accuracy when predicting game outcomes, especially for Player X and Player O wins. However, the model still struggles to correctly identify tie games, indicating the features or approach may not fully capture the complexity of this class. To address this, our next step will be to experiment with a decision tree classifier, which might better handle the nuances of tie predictions.

# Third Attempt

In [ ]:
# Import Libraries
from pyspark.ml.classification import DecisionTreeClassifier

# Get the list of columns representing moves (features)
move_columns = [col for col in df.columns if col.startswith('Move')]

In [ ]:
# Define Pipeline Stages

# Create stages for the pipeline
stages = []

# Convert Winner to numeric
labelIndexer = StringIndexer(inputCol="Winner", outputCol="label")
stages.append(labelIndexer)

# Assemble features
assembler = VectorAssembler(inputCols=move_columns, outputCol="features")
stages.append(assembler)

# Create and train the model
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", maxDepth=10)
stages.append(dt)

In [ ]:
# Create the pipeline
pipeline = Pipeline(stages=stages)

# Split the data into training and test sets
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
# Train model
model = pipeline.fit(trainingData)

# Make predictions
predictions = model.transform(testData)

In [ ]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

# Calculate accuracy on the test data
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.880362766749866


In [ ]:
# Get the label mapping
label_mapping = {float(idx): category for idx, category in enumerate(model.stages[0].labels)}
print("Label Mapping:", label_mapping)

# Print confusion matrix
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)

print("Confusion Matrix:")
print(metrics.confusionMatrix())

Label Mapping: {0.0: 'X', 1.0: 'O', 2.0: '-'}
Confusion Matrix:
DenseMatrix([[35339.,     0.,  4030.],
             [    0., 23486.,     0.],
             [ 5125.,     0.,  8543.]])


In [ ]:
# Print classification report
for label in sorted(label_mapping.keys()):
    print(f"\nClass {label_mapping[label]}:")
    print(f"Precision = {metrics.precision(label)}")
    print(f"Recall = {metrics.recall(label)}")
    print(f"F1 Score = {metrics.fMeasure(label)}")


Class X:
Precision = 0.8733442071965204
Recall = 0.8976351952043486
F1 Score = 0.8853231119962922

Class O:
Precision = 1.0
Recall = 1.0
F1 Score = 1.0

Class -:
Precision = 0.6794718841962937
Recall = 0.6250365817968979
F1 Score = 0.6511184787165123


In [ ]:
# Extract the trained Decision Tree model
dt_model = model.stages[-1]

# Print importance of each move feature
print("\nFeature Importances:")
for feature, importance in zip(move_columns, dt_model.featureImportances):
    print(f"{feature}: {importance}")


Feature Importances:
Move 1-X (Row-Col): 0.01382707668664977
Move 2-O (Row-Col): 0.016495663824717694
Move 3-X (Row-Col): 0.014594043536041459
Move 4-O (Row-Col): 0.008625303580357332
Move 5-X (Row-Col): 0.011254574258243395
Move 6-O (Row-Col): 0.04539013296737967
Move 7-X (Row-Col): 0.07180314877789555
Move 8-O (Row-Col): 0.47585735223813824
Move 9-X (Row-Col): 0.34215270413057697


### 🔧 Optimization Next Steps
Great news — our model is now doing way better at predicting ties, with solid precision, recall, and F1 scores around 0.65! That’s a big improvement.

Now, we’re curious to see if we can boost performance even more by letting our decision tree get a little deeper — increasing its max depth from 10 to 20 — to better capture the game’s complexity and nuances. Let’s try that next and see what kind of gains we can get!

# Fourth Attempt

In [ ]:
# Get the list of move columns
move_columns = [col for col in df.columns if col.startswith('Move')]

In [ ]:
# Create stages for the pipeline
stages = []

# Convert Winner to numeric
labelIndexer = StringIndexer(inputCol="Winner", outputCol="label")
stages.append(labelIndexer)

# Assemble features
assembler = VectorAssembler(inputCols=move_columns, outputCol="features")
stages.append(assembler)

# Create and train the model
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label", maxDepth=20)
stages.append(dt)

In [ ]:
# Create the pipeline
pipeline = Pipeline(stages=stages)

# Split the data into training and test sets
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
# Train model
model = pipeline.fit(trainingData)

# Make predictions
predictions = model.transform(testData)

In [ ]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9926427348640278


In [ ]:
# Get the label mapping
label_mapping = {float(idx): category for idx, category in enumerate(model.stages[0].labels)}
print("Label Mapping:", label_mapping)

Label Mapping: {0.0: 'X', 1.0: 'O', 2.0: '-'}


In [ ]:
# Print confusion matrix
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)

print("Confusion Matrix:")
print(metrics.confusionMatrix())

# Print classification report
for label in sorted(label_mapping.keys()):
    print(f"\nClass {label_mapping[label]}:")
    print(f"Precision = {metrics.precision(label)}")
    print(f"Recall = {metrics.recall(label)}")
    print(f"F1 Score = {metrics.fMeasure(label)}")

Confusion Matrix:
DenseMatrix([[39137.,     0.,   232.],
             [    0., 23486.,     0.],
             [  331.,     0., 13337.]])

Class X:
Precision = 0.9916134590047634
Recall = 0.9941070385328558
F1 Score = 0.9928586831056483

Class O:
Precision = 1.0
Recall = 1.0
F1 Score = 1.0

Class -:
Precision = 0.9829022035522146
Recall = 0.9757828504536142
F1 Score = 0.9793295884275067


In [ ]:
# Print feature importances
dt_model = model.stages[-1]
print("\nFeature Importances:")
for feature, importance in zip(move_columns, dt_model.featureImportances):
    print(f"{feature}: {importance}")


Feature Importances:
Move 1-X (Row-Col): 0.03932368747394982
Move 2-O (Row-Col): 0.05249921449997855
Move 3-X (Row-Col): 0.03130835596740921
Move 4-O (Row-Col): 0.04202429717300386
Move 5-X (Row-Col): 0.03697436458475791
Move 6-O (Row-Col): 0.06765544704979858
Move 7-X (Row-Col): 0.07763065507386602
Move 8-O (Row-Col): 0.39117996682469797
Move 9-X (Row-Col): 0.2614040113525382


### ✅ Optimized Model Results & Interpretation
After increasing our decision tree depth to 20, we saw a significant boost in performance — especially in identifying tied games, which had previously been a challenge. Our overall accuracy rose to 99.26%, and precision, recall, and F1 scores were excellent across all three classes:

- X Wins: F1 Score ≈ 0.99
- O Wins: F1 Score = 1.00
- Ties (-): F1 Score ≈ 0.98

The confusion matrix confirms that the model makes very few mistakes, with just a small number of ties misclassified as X wins. It’s not perfect, but it’s pretty close — and importantly, much better than our earlier attempts!

These results make sense in context: we're working with a dataset of complete, valid Tic-Tac-Toe games. Since Tic-Tac-Toe is a solved game with deterministic outcomes, our model is able to learn consistent patterns and predict outcomes with impressive accuracy.

This also highlights how powerful decision trees can be, especially when given enough depth to capture more complex patterns in the move sequences. While we’re not at 100%, we’re getting closer to creating a model that really "understands" how the game plays out — especially in recognizing those tricky ties.

# Data Exploration with SQL Queries

In [ ]:
# Number of games X won
spark.sql("""
    SELECT COUNT(*) as X_wins
    FROM tic_tac_toe
    WHERE Winner = 'X'
""").show()

+------+
|X_wins|
+------+
|131184|
+------+



In [ ]:
# Most common first move

spark.sql("""
    SELECT `Move 1-X (Row-Col)`, COUNT(*) as count
    FROM tic_tac_toe
    WHERE Winner = 'X'
    GROUP BY `Move 1-X (Row-Col)`
    ORDER BY count DESC
    LIMIT 1
""").show()

+------------------+-----+
|Move 1-X (Row-Col)|count|
+------------------+-----+
|               1-1|15648|
+------------------+-----+



In [ ]:
# X won in 5 moves

spark.sql("""
    SELECT *
    FROM tic_tac_toe
    WHERE Winner = 'X' AND `Move 6-O (Row-Col)` = '---'
""").show()

+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Winner|Move 1-X (Row-Col)|Move 2-O (Row-Col)|Move 3-X (Row-Col)|Move 4-O (Row-Col)|Move 5-X (Row-Col)|Move 6-O (Row-Col)|Move 7-X (Row-Col)|Move 8-O (Row-Col)|Move 9-X (Row-Col)|
+------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|     X|               0-0|               0-1|               1-0|               0-2|               2-0|               ---|               ---|               ---|               ---|
|     X|               0-0|               0-1|               1-0|               1-1|               2-0|               ---|               ---|               ---|               ---|
|     X|               0-0|               0-1|               1-0|               1-2|               2

In [ ]:
spark.sql("""
    SELECT
        COUNT(*) as games_count,
        COLLECT_LIST(
            CONCAT(
                `Move 1-X (Row-Col)`, ', ',
                `Move 2-O (Row-Col)`, ', ',
                `Move 3-X (Row-Col)`, ', ',
                `Move 4-O (Row-Col)`, ', ',
                `Move 5-X (Row-Col)`
            )
        ) as winning_sequences
    FROM tic_tac_toe
    WHERE Winner = 'X' AND `Move 6-O (Row-Col)` = '---'
""").show(truncate=False)

+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Games won by X
spark.sql("""
    SELECT
        CASE
            WHEN `Move 6-O (Row-Col)` = '---' THEN '5 moves'
            WHEN `Move 8-O (Row-Col)` = '---' THEN '7 moves'
            WHEN `Move 9-X (Row-Col)` != '---' THEN '9 moves'
        END AS moves_to_win,
        COUNT(*) as games_count
    FROM tic_tac_toe
    WHERE Winner = 'X'
    GROUP BY moves_to_win
    ORDER BY
        CASE moves_to_win
            WHEN '5 moves' THEN 1
            WHEN '7 moves' THEN 2
            WHEN '9 moves' THEN 3
        END
""").show()

+------------+-----------+
|moves_to_win|games_count|
+------------+-----------+
|     5 moves|       1440|
|     7 moves|      47952|
|     9 moves|      81792|
+------------+-----------+



In [ ]:
# Games won by each
spark.sql("""
    SELECT
        Winner,
        CASE
            WHEN `Move 2-O (Row-Col)` = '---' THEN 1
            WHEN `Move 3-X (Row-Col)` = '---' THEN 2
            WHEN `Move 4-O (Row-Col)` = '---' THEN 3
            WHEN `Move 5-X (Row-Col)` = '---' THEN 4
            WHEN `Move 6-O (Row-Col)` = '---' THEN 5
            WHEN `Move 7-X (Row-Col)` = '---' THEN 6
            WHEN `Move 8-O (Row-Col)` = '---' THEN 7
            WHEN `Move 9-X (Row-Col)` = '---' THEN 8
            ELSE 9
        END AS number_of_moves,
        COUNT(*) as games_count
    FROM tic_tac_toe
    GROUP BY Winner, number_of_moves
    ORDER BY Winner, number_of_moves
""").show(20)

+------+---------------+-----------+
|Winner|number_of_moves|games_count|
+------+---------------+-----------+
|     -|              9|      46080|
|     O|              6|       5328|
|     O|              8|      72576|
|     X|              5|       1440|
|     X|              7|      47952|
|     X|              9|      81792|
+------+---------------+-----------+

